In [ ]:
pip install keras_tuner

## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 528].index
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dtype='int')
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_test, X_train, y_test, y_train = train_test_split(X, y, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# nn = tf.keras.models.Sequential()

# nn = tf.keras.models.Sequential()
# nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=43))
# nn.add(tf.keras.layers.Dropout(0.2))
# nn.add(tf.keras.layers.Dense(units=64, activation='relu'))
# nn.add(tf.keras.layers.Dropout(0.2))
# nn.add(tf.keras.layers.Dense(units=32, activation='relu'))
# nn.add(tf.keras.layers.Dropout(0.2))
# nn.add(tf.keras.layers.Dense(units=1, activation='relu'))

# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# # Check the structure of the model
# nn.summary()

In [ ]:
import kerastuner as kt

# Define a model building function for Keras Tuner
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=256, step=32), activation='relu', input_dim=43))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Instantiate the tuner
tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=20, factor=3, directory='my_dir', project_name='intro_to_kt')

# Run the hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))


Trial 30 Complete [00h 01m 24s]
val_accuracy: 0.7294355630874634

Best val_accuracy So Far: 0.7294355630874634
Total elapsed time: 00h 15m 09s


In [ ]:
# Compile the model
# nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Use the Adam optimizer with a custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
268/268 [==============================] - 1s 5ms/step - loss: 0.5424 - accuracy: 0.7403
Epoch 2/100
268/268 [==============================] - 2s 7ms/step - loss: 0.5416 - accuracy: 0.7416
Epoch 3/100
268/268 [==============================] - 2s 9ms/step - loss: 0.5417 - accuracy: 0.7409
Epoch 4/100
268/268 [==============================] - 3s 10ms/step - loss: 0.5391 - accuracy: 0.7401
Epoch 5/100
268/268 [==============================] - 2s 7ms/step - loss: 0.5407 - accuracy: 0.7423
Epoch 6/100
268/268 [==============================] - 2s 6ms/step - loss: 0.5405 - accuracy: 0.7415
Epoch 7/100
268/268 [==============================] - 1s 4ms/step - loss: 0.5392 - accuracy: 0.7423
Epoch 8/100
268/268 [==============================] - 1s 3ms/step - loss: 0.5409 - accuracy: 0.7418
Epoch 9/100
268/268 [==============================] - 1s 3ms/step - loss: 0.5397 - accuracy: 0.7418
Epoch 10/100
268/268 [==============================] - 1s 2ms/step - loss: 0.5397 - accur

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

804/804 - 1s - loss: 0.5750 - accuracy: 0.7276 - 1s/epoch - 2ms/step
Loss: 0.5749703645706177, Accuracy: 0.7276084423065186


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# # Create a method that creates a new Sequential model with hyperparameter options
# def create_model(hp):
#     nn = tf.keras.models.Sequential()

#     # Allow kerastuner to decide which activation function to use in hidden layers
#     activation = hp.Choice('activation',['relu','tanh','sigmoid'])

#     # Allow kerastuner to decide number of neurons in first layer
#     nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
#         min_value=1,
#         max_value=10,
#         step=2), activation=activation, input_dim=43))

#     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#     for i in range(hp.Int('num_layers', 1, 6)):
#         nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
#             min_value=1,
#             max_value=10,
#             step=2),
#             activation=activation))

#     nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#     # Compile the model
#     nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

#     return nn

In [ ]:
# # Import the kerastuner library
# import keras_tuner as kt

# tuner = kt.Hyperband(
#     create_model,
#     objective="val_accuracy",
#     max_epochs=20,
#     hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
# # Run the kerastuner search for best hyperparameters
# tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 47 Complete [00h 00m 43s]
val_accuracy: 0.7290856838226318

Best val_accuracy So Far: 0.7290856838226318
Total elapsed time: 00h 20m 35s

Search: Running Trial #48

Value             |Best Value So Far |Hyperparameter
relu              |tanh              |activation
3                 |9                 |first_units
5                 |2                 |num_layers
3                 |3                 |units_0
1                 |9                 |units_1
7                 |7                 |units_2
5                 |9                 |units_3
7                 |9                 |units_4
5                 |3                 |units_5
20                |20                |tuner/epochs
7                 |7                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
2                 |2                 |tuner/round
0043              |0042              |tuner/trial_id

Epoch 8/20
268/268 [==============================] - 5s 14ms/step - loss: 0.5714 - acc

KeyboardInterrupt: 

In [ ]:
# # Get best model hyperparameters
# best_hyper = tuner.get_best_hyperparameters(1)[0]
# best_hyper.values

In [ ]:
# # Evaluate best model against full test data
# best_model = tuner.get_best_models(1)[0]
# model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# # Export our model to HDF5 file
# best_model.save("AlphabetSoupCharity_Optimization.h5")